### Importando librerias

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

%matplotlib inline

### Preparando los datos

In [2]:
# importing data
diamonds =pd.read_csv('diamonds.csv')
diamonds.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [3]:
diamonds=pd.concat([diamonds, pd.get_dummies(diamonds['cut'], prefix='cut', drop_first=True)],axis=1)
diamonds=pd.concat([diamonds, pd.get_dummies(diamonds['color'], prefix='cut', drop_first=True)],axis=1)
diamonds=pd.concat([diamonds, pd.get_dummies(diamonds['clarity'], prefix='cut', drop_first=True)],axis=1)
diamonds.drop(['cut', 'color', 'clarity'], axis=1, inplace=True)

In [4]:
diamonds.head()

,Unnamed: 0,carat,depth,table,price,x,y,z,cut_Good,cut_Ideal,...,cut_H,cut_I,cut_J,cut_IF,cut_SI1,cut_SI2,cut_VS1,cut_VS2,cut_VVS1,cut_VVS2
0,1,0.23,61.5,55.0,326,3.95,3.98,2.43,False,True,...,False,False,False,False,False,True,False,False,False,False
1,2,0.21,59.8,61.0,326,3.89,3.84,2.31,False,False,...,False,False,False,False,True,False,False,False,False,False
2,3,0.23,56.9,65.0,327,4.05,4.07,2.31,True,False,...,False,False,False,False,False,False,True,False,False,False
3,4,0.29,62.4,58.0,334,4.20,4.23,2.63,False,False,...,False,True,False,False,False,False,False,True,False,False
4,5,0.31,63.3,58.0,335,4.34,4.35,2.75,True,False,...,False,False,True,False,False,True,False,False,False,False


### Preparando objeto para modelación

In [5]:
from sklearn.preprocessing import RobustScaler
traget_name='price'
robust_scaler=RobustScaler()
X=diamonds.drop('price', axis=1)
X=robust_scaler.fit_transform(X)
y=diamonds[traget_name]

### Comparando modelos usando Cross-validation

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
#from xgboost.sklearn import DecisionTreeRegressor
from xgboost.sklearn import XGBRegressor

import xgboost as xgb
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor


### La funcion cross-validation y la evaluacion de metricas multiples

In [7]:
#XGBoost

xgb=XGBRegressor(random_state=2, reg_alpha=0.005, gama=0.1, subsample=0.7,
                 colsample_bytree=0.9, colsample_bylevel=0.9, colsample_bynode=0.9)

In [12]:

xgb_test_mse = cross_validate(estimator=xgb, X=X, y=y, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)['test_score']

In [28]:
tree=DecisionTreeRegressor(min_samples_leaf=.0001)
tree_test_mse=cross_validate(estimator=tree, X=X, y=y,
                                scoring='neg_mean_squared_error',
                                cv=10, n_jobs=-1)['test_score']

In [33]:
Ex_tree=ExtraTreesRegressor(n_estimators=100)
Extra_Tree_test_mse=cross_validate(estimator=Ex_tree, X=X, y=y, scoring='neg_mean_squared_error',
                                    cv=10, n_jobs=-1) ['test_score']

In [14]:
#Random Forest

RF=RandomForestRegressor(n_estimators=50, max_depth=16, random_state=55, n_jobs=-1)
RF_test_mse=cross_validate(estimator=RF, X=X, y=y, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)['test_score']


In [20]:
base_estimator = DecisionTreeRegressor()

# Create a BaggingRegressor using the decision tree as the base estimator
bagging = BaggingRegressor(base_estimator=base_estimator, n_estimators=15, max_features=0.75, random_state=55, n_jobs=-1)
bagging_test_mse=cross_validate(estimator=boosting, X=X, y=y, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)['test_score']

In [17]:
#boosting

boosting=AdaBoostRegressor(n_estimators=50, learning_rate=0.05, random_state=55)
boosting_test_mse=cross_validate(estimator=boosting, X=X, y=y, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)['test_score']

In [ ]:
mse_modelos=- 1 * pd.DataFrame({'xgboost': xgb_test_mse, 
                                'Arbol_Decision': tree_test_mse, 
                                'Extra Tree': Extra_Tree_test_mse, 
                                'Bagging': bagging_test_mse,
                                #'RandomForest': RandomForest_test_mse,
                                'Boosting': boosting_test_mse})

In [34]:
import pandas as pd

data_dict = {
    'xgboost': pd.Series(xgb_test_mse),
    'Arbol_Decision': pd.Series(tree_test_mse), 
    'Extra Tree': pd.Series(Extra_Tree_test_mse),
    'Bagging': pd.Series(bagging_test_mse),
    'Boosting': pd.Series(boosting_test_mse)
}

mse_modelos = -1 * pd.DataFrame(data_dict)


In [35]:
mse_modelos

,xgboost,Arbol_Decision,Extra Tree,Bagging,Boosting
0,2.958016e+05,3.811463e+05,367130.489664,1.490785e+06,1.490785e+06
1,1.394889e+05,8.875724e+04,2814.885826,8.661486e+05,8.661486e+05
2,2.264584e+05,2.053015e+05,28766.164151,2.112730e+06,2.112730e+06
3,8.853517e+05,8.322079e+05,54436.810342,3.967516e+06,3.967516e+06
4,4.852958e+06,4.603124e+06,652306.941674,1.026740e+07,1.026740e+07
5,2.155779e+05,1.367511e+05,146134.715489,2.056105e+06,2.056105e+06
6,7.453715e+03,8.352263e+03,8095.366816,3.735307e+04,3.735307e+04
7,1.918280e+04,2.611168e+04,5642.467070,4.878356e+05,4.878356e+05
8,2.425234e+04,4.005390e+04,4736.882572,5.861842e+05,5.861842e+05
9,2.052634e+05,1.771575e+05,228543.398058,2.469071e+06,2.469071e+06


In [36]:
mse_modelos.mean().sort_values()

Extra Tree        1.498608e+05
Arbol_Decision    6.498963e+05
xgboost           6.871789e+05
Bagging           2.434113e+06
Boosting          2.434113e+06
dtype: float64

In [39]:
fig=px.bar(x=['Arbol_Decision', 'Extra Tree', 'Bagging', 'RandomForest', 'Boosting'],
            y=[mse_modelos.mean()],
            color=['Arbol_Decision', 'Extra Tree', 'Bagging', 'RandomForest','Boosting'],
            labels={'x':'Model', 'y':'Accuracy'},
            title='Validacion de modelos con k-fold cross validation')
fig.show()